#Grupo 9 Práctica 2

#Prerequisitos:

##Import de librerías necesarias

In [5]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install scikit-optimize
!pip3 install skopt
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install statsmodels
!pip3 install lightgbm
!pip3 install xgboost

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import ensemble, tree, linear_model
import time
from sklearn import metrics
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Integer
from sklearn.pipeline import Pipeline
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score, balanced_accuracy_score, confusion_matrix



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement skopt (from versions: none)
ERROR: No matching distribution found for skopt

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\aalex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


##Extracción de datos

In [4]:
datos = pd.read_pickle('./attrition_available_9.pkl')
target = datos['Attrition']
data = datos.drop(columns=['Attrition'])
ds = data.T
X = [] 
y = target.T.tolist()
for i in ds.columns:
    X.append(ds[i].tolist())


#EDA

In [ ]:
print(datos.head())
print(datos.info())
print(datos['Attrition'].value_counts(normalize=True))
print(datos.isnull().sum())

datos_no_na = datos.dropna()
datos_no_na.fillna(datos_no_na['cat_var'].mode()[0], inplace=True)
datos_no_na.fillna(datos_no_na['num_var'].median(), inplace=True)

pos_class = datos_no_na[datos_no_na['Attrition'] == 1]
neg_class = datos_no_na[datos_no_na['Attrition'] == 0]

pos_class_upsampled = pos_class.sample(pos_class, n=len(neg_class), replace=True, random_state=42)

datos_balanced = pd.concat([pos_class_upsampled, neg_class])
datos_balanced = datos_balanced.sample(frac=1, random_state=42)

sns.pairplot(datos_balanced, hue='Attrition', palette='Set1')
plt.show()

sns.boxplot(x='Attrition', y='Age', data=datos_balanced)
plt.show()

sns.countplot(x='cat_var', hue='Attrition', data=datos_balanced)
plt.show()

Sabemos que se trata de un problema de clasificación desbalanceado, por lo que se debe tener en cuenta a la hora de evaluar los modelos. Además, se debe tener en cuenta que el objetivo es predecir si un empleado se va a ir de la empresa o no, por lo que se debe tener en cuenta que la métrica a utilizar es la precisión. Missing values, se debe imputar los datos, y se debe utilizar una partición estratificada. 

#Construcción de modelos

##Extracción del conjunto de datos train/test

In [ ]:
X_train = X[0:2940]
X_test = X[2940:]
y_train = y[0:2940]
y_test = y[2940:]
X_train_train = X_train[0:1960]
X_train_test = X_train[1960:]
y_train_train = y_train[0:1960]
y_train_test = y_train[1960:]

##Modelo 1: Regresión Logística

In [ ]:
#--------------------------------------
# Logistic Regression
#--------------------------------------

inicio = time.time()
scaler = None
clf = LogisticRegression(random_state=9, class_weight='balanced').fit(X_train_train, y_train_train)
y_pred = clf.predict(X_train_test)
f1 = f1_score(y_train_test, y_pred)
bal_acc = balanced_accuracy_score(y_train_test, y_pred)
cm = confusion_matrix(y_train_test, y_pred)
final = time.time()
tiempo = final - inicio
print(f"El tiempo tardado es de {tiempo} segundos.\n")
print(f"El f1 es de {f1}\n")
print(f"El balanced accuracy es de {bal_acc}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Logistic Regression
#--------------------------------------

##Modelo 2: Boosting

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------



#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

##Modelo 2.1: LightGBM

In [ ]:
#--------------------------------------
# Boosting with LightGBM
#--------------------------------------



#--------------------------------------
# Boosting with LightGBM
#--------------------------------------

##Modelo 2.2: XGBoost

In [ ]:
#--------------------------------------
# Boosting with XGBoost
#--------------------------------------



#--------------------------------------
# Boosting with XGBoost
#--------------------------------------

#Modelo con atributos de tipo filtro

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------



#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------



#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with XGBoost with filter
#--------------------------------------



#--------------------------------------
# Boosting with XGBoost with filter
#--------------------------------------

#Modelos con EDA

##Modelo 1: Regresión Logística

In [ ]:
#--------------------------------------
# Logistic Regression after EDA
#--------------------------------------



#--------------------------------------
# Logistic Regression after EDA
#--------------------------------------

##Modelo 2: Boosting

In [ ]:
#--------------------------------------
# Boosting with (...) after EDA
#--------------------------------------



#--------------------------------------
# Boosting with (...) after EDA
#--------------------------------------

#Conclusiones y comparación de modelos

#Modelo final